# Skills gaps analysis

Here, we demonstrate our approach for identifying the most prevalent gaps across multiple transitions. This information could inform upskilling efforts and training provision for a broader group of workers.

To demonstrate this approach, we assessed the typical skills gaps in transitions for workers leaving high-risk occupations within the sales and services skills-based sector. We differentiated between (1) pathways that left workers within (safer) sales and services occupations and (2) pathways that moved them into the business and administration skills-based sector.

# 0. Import dependencies and inputs

In [1]:
%run ../notebook_preamble_Transitions.ipy
import os

data = load_data.Data()
sims = load_data.Similarities()

In [2]:
file_name = 'filter_matrices_Report_occupations.pickle'
export_path = f'{data_folder}interim/transitions/{file_name}'

if os.path.exists(export_path):
    filter_matrices = pickle.load(open(export_path,'rb'))
    print(f'Imported filtering matrices from {export_path}')


Imported filtering matrices from /Users/karliskanders/Documents/mapping-career-causeways/codebase/data/interim/transitions/filter_matrices_Report_occupations.pickle


In [3]:
pd.set_option('max_colwidth', 200)

# 1. Skills gap analysis for 'high risk' sales and services occupations


## Generate the transitions to analyse

In [4]:
# Find 'high risk' occupation IDs
high_risk_ids = data.occ_report[data.occ_report.risk_category=='High risk'].id.to_list()
# Find safe and desirable transitions from 'high risk' occupations
transition_df = trans_utils.get_transitions(origin_ids=high_risk_ids, destination_ids='report')
trans_to_analyse = transition_df[transition_df.is_strictly_safe_desirable].reset_index(drop=True)


In [5]:
len(trans_to_analyse)

3619

In [6]:
# Initialise a class for assessing skills gaps
# (might take a couple minutes for 1000s of transitions; it is calculating detailed skills similarities)
gaps = trans_utils.SkillsGaps(trans_to_analyse)

Time elapsed: 178.69 sec ( 0.049 per transition)


In [7]:
# Add sector names to the transitions dataframe
trans_sector = data.add_field_to_occupation(trans_to_analyse, 'origin_id', 'skills_based_sector', 'origin')
trans_sector = data.add_field_to_occupation(trans_sector, 'destination_id', 'skills_based_sector', 'destination')
# Select transitions from sales and services workers
trans_sector = trans_sector[trans_sector.origin_skills_based_sector == 'sales & services workers']
len(trans_sector)

538

## 1.1 Desirable transitions to safer sales and services occupations 

In [8]:
trans = trans_sector[trans_sector.destination_skills_based_sector == 'sales & services workers']
print(len(trans))

348


In [9]:
# Setup the skills gaps analysis by specifying which transitions and which skills match to analyse
gaps.setup(transition_indices=trans.index.to_list(), skills_type='destination')

In [10]:
# Find the most prevalent gaps (95th percentile) at the individual skills level
gaps.prevalent_skills_gaps(top_x=95, percentile=True)

,id,preferred_label,level_1,level_2,level_3,counts,prevalence,score,stdev
242,6288,maintain customer service,A1,A1.12,A1.12.3,106,0.305,0.860519,0.253998
212,5558,identify customer's needs,S1,S1.7,S1.7.2,86,0.247,0.780419,0.379114
27,710,communicate with customers,S3,S3.4,S3.4.1,100,0.287,0.776270,0.323304
81,1809,maintain relationship with customers,S1,S1.2,S1.2.3,112,0.322,0.770143,0.313632
0,31,guarantee customer satisfaction,A1,A1.12,A1.12.3,73,0.210,0.763342,0.301236
125,3248,maintain relationship with suppliers,S1,S1.2,S1.2.3,64,0.184,0.667234,0.374838
42,1020,implement marketing strategies,S1,S1.6,S1.6.2,61,0.175,0.650131,0.432875
195,5206,assist customers,S3,S3.4,S3.4.4,45,0.129,0.624533,0.398101
402,9777,implement sales strategies,S1,S1.6,S1.6.2,62,0.178,0.590452,0.445233
516,12494,monitor customer service,S4,S4.8,S4.8.3,43,0.124,0.555372,0.403384


In [11]:
# Find most prevalent gaps in terms of Level-3 skills cluster categories
gaps.prevalent_cluster_gaps('level_3', top_x=90, percentile=True)


,code,title,counts,prevalence,score,stdev,skills
3,A1.12.3,provide high quality client service,276,0.793,0.665083,0.406479,"maintain customer service (0.86), guarantee customer satisfaction (0.76), provide customer follow-up services (0.48)"
56,S1.2.3,developing professional relationships or networks,269,0.773,0.627955,0.402770,"maintain relationship with customers (0.77), maintain relationship with suppliers (0.67), build business relationships (0.71)"
70,S1.7.2,engaging with others to identify needs,113,0.325,0.605876,0.464114,"identify customer's needs (0.78), perform customers’ needs analysis (0.08), initiate contact with buyers (0.01)"
68,S1.6.2,"promoting products, services, or programs",188,0.540,0.520245,0.444411,"implement sales strategies (0.59), implement marketing strategies (0.65), supervise merchandise displays (0.6)"
108,S3.4.1,providing information to the public and clients,190,0.546,0.493326,0.444010,"communicate with customers (0.78), provide information (0.01), answer incoming calls (0.22)"
26,K4.1.6,wholesale and retail sales,237,0.681,0.360418,0.448918,"characteristics of services (0.46), product comprehension (0.44), sales activities (0.01)"
110,S3.4.4,providing general assistance to people,231,0.664,0.276307,0.426300,"greet guests (0.38), assist customers (0.62), assist guest departure (0.0)"
102,S3.3.1,complying with health and safety procedures,108,0.310,0.276167,0.425201,"comply with food safety and hygiene (0.45), apply health and safety standards (0.28), manage health and safety standards (0.16)"
67,S1.6.1,selling products or services,159,0.457,0.223742,0.379117,"maximise sales revenues (0.13), set sales promotions (0.0), work independently in sales (0.0)"
43,S1.1.1,negotiating and managing contracts and agreements,172,0.494,0.211157,0.337594,"negotiate sales contracts (0.35), negotiate buying conditions (0.24), manage contracts (0.29)"


The last column indicates the three most prevalent skills in the cluster, and their average matching scores across all transitions where they are required in the destination occupations.

## 1.2 Desirable transitions to safer business and administration roles

In [12]:
trans = trans_sector[trans_sector.destination_skills_based_sector == 'business & administration workers']
print(len(trans))

# Setup the skills gaps analysis by specifying which transitions and which skills match to analyse
gaps.setup(transition_indices=trans.index.to_list(), skills_type='destination')

135


In [13]:
# Find the most prevalent gaps at the individual skills level
gaps.prevalent_skills_gaps(top_x=95, percentile=True)

,id,preferred_label,level_1,level_2,level_3,counts,prevalence,score,stdev
306,6288,maintain customer service,A1,A1.12,A1.12.3,12,0.089,0.998000,0.000000
46,1020,implement marketing strategies,S1,S1.6,S1.6.2,12,0.089,0.809000,0.353040
168,3475,use different communication channels,S1,S1.2,S1.2.1,12,0.089,0.748500,0.451362
30,710,communicate with customers,S3,S3.4,S3.4.1,14,0.104,0.744286,0.344880
555,11596,build business relationships,S1,S1.2,S1.2.3,12,0.089,0.735417,0.431149
265,5558,identify customer's needs,S1,S1.7,S1.7.2,26,0.193,0.693654,0.409201
463,9777,implement sales strategies,S1,S1.6,S1.6.2,12,0.089,0.683167,0.465156
630,13387,perform market research,S2,S2.1,S2.1.1,14,0.104,0.648714,0.486542
39,832,comply with food safety and hygiene,S3,S3.3,S3.3.1,23,0.170,0.620348,0.485125
109,2187,handle customer complaints,S1,S1.1,S1.1.3,22,0.163,0.500909,0.508793


In [14]:
# Find most prevalent gaps in terms of Level-3 skills cluster categories
gaps.prevalent_cluster_gaps('level_3', top_x=90, percentile=True)


,code,title,counts,prevalence,score,stdev,skills
80,S1.7.2,engaging with others to identify needs,44,0.326,0.504250,0.415309,"identify customer's needs (0.69), perform customers’ needs analysis (0.2), persuade clients with alternatives (0.38)"
77,S1.6.2,"promoting products, services, or programs",45,0.333,0.442933,0.485451,"implement marketing strategies (0.81), implement sales strategies (0.68), promote sustainable energy (0.5)"
61,S1.2.3,developing professional relationships or networks,87,0.644,0.416862,0.416976,"develop professional network (0.26), build business relationships (0.74), maintain relationship with suppliers (0.88)"
145,S4.8.1,supervising a team or group,66,0.489,0.382515,0.449037,"manage staff (0.45), coordinate activities across hospitality rooms division (0.0), lead a team (0.36)"
136,S4.3.1,managing budgets or finances,72,0.533,0.321056,0.467111,"manage budgets (0.49), manage accounts (0.01), manage hospitality revenue (0.01)"
60,S1.2.1,coordinating activities with others,93,0.689,0.175204,0.325881,"liaise with managers (0.03), ensure cross-department cooperation (0.08), use different communication channels (0.75)"
127,S4.1.2,"developing financial, business or marketing plans",48,0.356,0.170479,0.308652,"strive for company growth (0.11), develop business case (0.02), forecast occupancy demand (0.0)"
106,S2.7.4,analysing business operations,70,0.519,0.126400,0.303496,"measure customer feedback (0.64), analyse business objectives (0.0), analyse supply chain trends (0.09)"
29,K4.1.4,marketing and advertising,67,0.496,0.125537,0.263639,"advertising techniques (0.21), ICT market (0.0), marketing principles (0.53)"
134,S4.2.2,planning and scheduling events and activities,84,0.622,0.119774,0.271697,"plan events (0.25), coordinate events (0.09), fix meetings (0.0)"
